In [15]:
import numpy as np
import pytreenet as ptn
from copy import deepcopy
from scipy.linalg import expm
from pytreenet.util import copy_object

In [3]:
def A_prime_dict(mps, mps_tilde):
    """"
    Function to build the A_prime matrixes in A.2
    Args:
        mps: MPS object
        mps_tilde: MPS object
    Returns:
        tensors_prime: Dictionary with the A_prime matrices
    """
    tensors = []
    tensors_prime = {}

    # Iterate over the sites
    for i in range(len(mps.nodes) - 1):
        # Extract the tensors
        A1 = mps.tensors[f"site{i}"][..., 0]
        A1_tilde = mps_tilde.tensors[f"site{i}"][..., 0]
        A2 = mps.tensors[f"site{i}"][..., 1]
        A2_tilde = mps_tilde.tensors[f"site{i}"][..., 1]

        # Combine the tensors
        tensors.append(A1 + A2)
        tensors.append(A1_tilde + A2_tilde)

        # Create the combined tensor
        A_0 = np.block([[A1, np.zeros_like(A1)], [np.zeros_like(A1), A1_tilde]])
        A_1 = np.block([[np.zeros_like(A2), A2], [A2_tilde, np.zeros_like(A2)]])
        A_prime = A_0 + A_1
        tensors_prime[mps.nodes[f"site{i}"].identifier] = A_prime

    # Handle the last site
    i = len(mps.nodes) - 1
    C1 = mps.tensors[f"site{i}"][..., 0].reshape(mps.tensors[f"site{i}"][..., 0].shape[0], 1)
    C1_tilde = mps_tilde.tensors[f"site{i}"][..., 0].reshape(mps_tilde.tensors[f"site{i}"][..., 0].shape[0], 1)
    C2 = mps.tensors[f"site{i}"][..., 1].reshape(mps.tensors[f"site{i}"][..., 1].shape[0], 1)
    C2_tilde = mps_tilde.tensors[f"site{i}"][..., 1].reshape(mps_tilde.tensors[f"site{i}"][..., 1].shape[0], 1)

    tensors.append(C1 + C2)
    tensors.append(C1_tilde + C2_tilde)

    C_0 = np.block([[C1], [C1_tilde]])
    C_1 = np.block([[C2], [C2_tilde]])
    C_prime = C_0 + C_1
    tensors_prime[mps.nodes[f"site{i}"].identifier] = C_prime

    return tensors_prime

In [4]:
##------------------------------------##
## Test the functions : normalize_ttn ##
##------------------------------------##

## MPS ##
shapes = [(5, 2), (5, 7, 2), (7, 3, 2), (3, 6, 2), (6, 30, 2), (30, 2)]
tensors1 = [ptn.crandn(shape) for shape in shapes]
mps1 = ptn.MatrixProductState.from_tensor_list(tensors1,root_site=5,node_prefix="site")

print( ptn.contract_two_ttns(mps1,mps1.conjugate()))
mps1_normalized = mps1.normalize_ttn(to_copy = False) # default is False
print( ptn.contract_two_ttns(mps1,mps1.conjugate()))
print(ptn.contract_two_ttns(mps1_normalized,mps1_normalized.conjugate()))

## TTN ##
ttn1 = ptn.random_big_ttns_two_root_children()

print("contraction befor normalization : " , ptn.contract_two_ttns(ttn1,ttn1.conjugate()))
ttn1_normalized = ttn1.normalize_ttn(to_copy = True)
print( ptn.contract_two_ttns(ttn1,ttn1.conjugate()))
print( ptn.contract_two_ttns(ttn1_normalized,ttn1_normalized.conjugate()))

(739827.5689470959+5.820766091346741e-11j)
(1+4.163336342344337e-17j)
(1+4.163336342344337e-17j)
contraction befor normalization :  (2655.6863640738857-5.684341886080802e-14j)
(2655.6863640738857-5.684341886080802e-14j)
(1.0000000000000002+0j)


In [5]:
ttn1 = ptn.random_big_ttns_two_root_children()
ttn2 = ptn.random_big_ttns_two_root_children()
##---------------------##
## Full density tensor ##
##---------------------##

## For pure states : (density tensor)^2 = density tensor 
pho , order = ttn1.density_tensor() 
# order = ['site0', 'site1', 'site2', 'site3', 'site4', 'site5', 'site6', 'site7']
# density tensor * density tensor = density tensor :
tensor = np.tensordot(pho,pho,axes=((8,9,10,11,12,13,14,15),(0,1,2,3,4,5,6,7)))
print(np.allclose(tensor,pho))

## trace of density tensor = 1
def tensor_trace(pho):
    """"
    Args :
        pho : Tensor
    Returns :
        trace : float
    """
    for _ in range(pho.ndim//2):
        pho = np.trace(pho, axis1 = 0, axis2 = pho.ndim//2)
    return pho

print(np.allclose(tensor_trace(pho),1))

##---------------------------##
## Full density tensor(ttno) ##
##---------------------------##

ttn = ptn.random_big_ttns_two_root_children()
pho_ttno = ttn.density_ttno()

## trace of density tensor = 1
def ttno_trace(pho):
    ttno_cct = deepcopy(pho)
    ttno_cct = ttno_cct.completely_contract_tree()[0] 
    for _ in range(ttno_cct.ndim//2):
        ttno_cct = np.trace(ttno_cct, axis1 = 0, axis2 = 1)
    return ttno_cct

print(np.allclose(ttno_trace(pho_ttno),1))

## check density_ttno and density_tensor compatibility : 
ttn = ptn.random_small_ttns()
pho , order = ttn.density_tensor() 
pho_ttno = ttn.density_ttno()
np.allclose(pho_ttno.completely_contract_tree()[0],pho.transpose(0,3,1,4,2,5))

##------------------------##
## Reduced density tensor ##
##------------------------##

ttn = ptn.random_big_ttns_two_root_children()
node_id = 'site0'
# first method :
pho_ref_1 = ttn.reduced_density_matrix_dict()[node_id]
pho_ref_2 = ttn.reduced_density_matrix(node_id)
# second method :
pho = ttn.reduced_density_matrix_2(node_id)

# trace of reduced density tensor = 1
print(np.trace(pho, axis1 = 0, axis2 = 1))
print(np.trace(pho_ref_1, axis1 = 0, axis2 = 1))
print(np.trace(pho_ref_2, axis1 = 0, axis2 = 1))

# Check Hermiticity : 
print(np.allclose(pho,pho.conjugate().T))
print(np.allclose(pho_ref_1,pho_ref_1.conjugate().T))
print(np.allclose(pho_ref_2,pho_ref_2.conjugate().T))

# calculate explicitly the reduced density tensor of a node in a tree tensor network
node_id = 'site4'
ttn = ttn.normalize_ttn(to_copy=True)
ttn.canonical_form(node_id , ptn.SVDParameters())
C = ttn.tensors[node_id]
reduced_density_matrix_dir = ptn.compute_transfer_tensor(C, tuple(range(C.ndim - 1)))


reduced_density_matrix_ref_1 = ttn.reduced_density_matrix_dict()[node_id]
reduced_density_matrix_ref_2 = ttn.reduced_density_matrix(node_id)

print(np.allclose(reduced_density_matrix_dir,reduced_density_matrix_ref_1))
print(np.allclose(reduced_density_matrix_dir,reduced_density_matrix_ref_2))


True
True
True
(0.9999999999999998+0j)
(0.9999999999999989+4.490731800328466e-18j)
(0.9999999999999958+9.583479375679694e-18j)
True
True
True


True
True


In [6]:
"""
First I did it with reduced_density_matrix_2, but I guess completely_contract_tree is more 
costly that moving the canoical center. I wonder why the results are
not same!
"""
ttn = ptn.random_big_ttns_two_root_children()
node_id = 'site2'
pho = ttn.reduced_density_matrix_2(node_id)
pho_tilde = ttn.reduced_density_matrix_dict()[node_id]
print(np.allclose(pho,pho_tilde)) ######### Fails

False


In [7]:
##---------------------##
## canonical_form :svd ##
##---------------------##

### MPS ###
shapes = [(5, 2), (5, 7, 2), (7, 3, 2), (3, 6, 2), (6, 30, 2), (30, 2)]
tensors1 = [ptn.crandn(shape) for shape in shapes]
mps1 = ptn.MatrixProductState.from_tensor_list(tensors1,root_site=5,node_prefix="site")

## test canonical_form for "site2" :
ref_contracted = ptn.contract_two_ttns(mps1,mps1.conjugate())
mps1.canonical_form("site2", ptn.SVDParameters())
direct_contracted = ptn.compute_transfer_tensor(mps1.tensors["site2"], tuple(range(mps1.tensors["site2"].ndim )))
print(np.allclose(ref_contracted, direct_contracted))

### TTN ###
ttn1 = ptn.random_big_ttns_two_root_children()

# test canonical_form for "site3"
direct_contracted = ptn.contract_two_ttns(ttn1,ttn1.conjugate())
ttn1.canonical_form("site3", ptn.SVDParameters())
ref_contracted = ptn.compute_transfer_tensor(ttn1.tensors["site3"], tuple(range(ttn1.tensors["site3"].ndim )))
print(np.allclose(ref_contracted, direct_contracted))


# Check canonical_form with QR and SVD compatibility 
ttn = ptn.random_big_ttns_two_root_children()
ttn1 = deepcopy(ttn)
ttn1.canonical_form('site0', ptn.SVDParameters(max_bond_dim = np.inf, rel_tol= -np.inf, total_tol=-np.inf))
ttn2 = deepcopy(ttn)
ttn2.canonical_form('site0')
a = ptn.compute_transfer_tensor(ttn1.tensors['site0'], (0,1,2))
b = ptn.compute_transfer_tensor(ttn2.tensors['site0'], (0,1,2))
c = ptn.contract_two_ttns(ttn,ttn.conjugate())
print(np.allclose(a,b))
print(np.allclose(a,c))
print(np.allclose(b,c))

True
True
True
True
True


In [14]:
##-------------------------##
## complete_canonical_form ##
##-------------------------##

# I defined new function because the canonical_form transforms ttns only to site cannonical form.
#    - complete_canonical_form transforms all tensors to left Isometry 
#      with respect to root site controlled by SVD truncation parameters.

### MPS ###
shapes = [(5, 2), (5, 7, 2), (7, 3, 2), (3, 6, 2), (6, 30, 2), (30, 2)]
tensors1 = [ptn.crandn(shape) for shape in shapes]
mps1 = ptn.MatrixProductState.from_tensor_list(tensors1,root_site=5,node_prefix="site")


## test complete_canonical_form : 
mps1_norm = ptn.contract_two_ttns(mps1,mps1.conjugate())
_ , norm = mps1.complete_canonical_form(ptn.SVDParameters())
    # check the norm 
print(np.allclose(mps1_norm,norm))
print(np.allclose(ptn.contract_two_ttns(mps1,mps1.conjugate()),1))
    # check the orthogonality of root site = "site5"
Identity = ptn.compute_transfer_tensor(mps1.tensors[mps1.root_id], (0,1) )
print(np.allclose(Identity,1))
    # check the orthogonality of "site3"
Identity = ptn.compute_transfer_tensor(mps1.tensors["site3"], (1,2) )
print(np.allclose(Identity,np.eye(6)))
    # check the orthogonality of leaf site = "site0"
print(np.allclose(ptn.compute_transfer_tensor(mps1.tensors["site0"], (1,) ),np.eye(2)))


### TTN ###
ttn1 = ptn.random_big_ttns_two_root_children()



## test complete_canonical_form :
ttn1_norm = ptn.contract_two_ttns(ttn1,ttn1.conjugate())
_ , norm = ttn1.complete_canonical_form(ptn.SVDParameters())
    # check the norm 
print(np.allclose(ttn1_norm,norm))
print(np.allclose(ptn.contract_two_ttns(ttn1,ttn1.conjugate()),1)) 
    # check the orthogonality of root site = "site0"
print(np.allclose(ptn.compute_transfer_tensor(ttn1.tensors[ttn1.root_id], (0,1,2) ),1))
    # check the orthogonality of leaf site = "site4"
print(np.allclose(ptn.compute_transfer_tensor(ttn1.tensors["site4"], (1,) ),np.eye(2)))
    # check the orthogonality of "site3"
Identity = ptn.compute_transfer_tensor(ttn1.tensors["site3"], (1,2,3) )
print(np.allclose(Identity,np.eye(2)))


True
True
True
True
True
(0.9999999999999998+0j)
True
True
True
True
True


In [9]:
tp = ptn.random_tensor_product(mps1, num_operators= len(mps1))
print(tp)
H = ptn.Hamiltonian(tp)
hamiltonian = ptn.TTNO.from_hamiltonian(H,mps1)


{'site1': array([[ 0.25397585-0.37323946j, -0.26691782-0.23933904j],
       [ 0.68601921-1.30767138j, -0.60245601+1.12681351j]]), 'site2': array([[-2.02941614-0.30079625j, -0.66850564-0.5442636j ],
       [-0.18835916+0.67173573j,  0.2762695 +0.50995177j]]), 'site4': array([[ 0.08226436-0.87566713j, -0.21475914-0.75201694j],
       [-0.16243023-1.24283638j, -0.30813163+0.10954597j]]), 'site3': array([[ 0.90729137+0.25735946j,  0.45024987+0.12249447j],
       [-0.97081991+1.08448584j, -0.57950414+0.24301743j]]), 'site5': array([[0.18882298-0.75678626j, 0.0873076 +0.42551639j],
       [1.36459378+0.43842693j, 0.21132063+0.50319474j]]), 'site0': array([[ 0.4361642 -1.33856301j, -0.31341537+0.72287677j],
       [ 0.15550583+1.09071626j,  0.24883691-0.34280232j]])}


TypeError: unhashable type: 'numpy.ndarray'

# Krylov space 

In [ ]:
"""
In first method, I applied one site projector at ecah step(along the same path as tdvp)
"""
# Initialize a random mps and ttn and their random hamiltonian : 

## MPS ##
"""
0 --- 1 --- 2 --- 3 --- 4 ---- 5
|     |     |     |     |      |

# order of legs =  [right_child, left_child, open_leg]
"""
shapes = [(5, 2), (5, 7, 2), (7, 3, 2), (3, 6, 2), (6, 30, 2), (30, 2)]
tensors1 = [ptn.crandn(shape) for shape in shapes]
mps1 = ptn.MatrixProductState.from_tensor_list(tensors1,root_site=5,node_prefix="site")

"""
Question : 
tp = ptn.random_tensor_product(mps1, num_operators= len(mps1))
H = ptn.Hamiltonian(tp)
hamiltonian = ptn.TTNO.from_hamiltonian(H,mps1)
this method no longer works (Error: state_diagram.obtain_tensor_shape)
"""
tensor = ptn.crandn([2,2,2,2,2,2,
                     2,2,2,2,2,2])
leg_dict = {"site0": 0, "site1": 1, "site2": 2, "site3": 3, "site4": 4, "site5": 5}
ham_mps = ptn.TTNO.from_tensor(mps1, tensor, leg_dict)


## start ##
Krylov_space = ptn.Krylov(mps1, ham_mps)
results = Krylov_space.run(num_steps=4)
print(results[0] == Krylov_space.initial_state)
print(len(results) == 5)
print(results[0] == Krylov_space.initial_state)
# Krylov_space.results[1] = P*hamiltonian |inisial_state>
# Krylov_space.results[2] = P*hamiltonian^2 |Krylov_space.results[1]>
# Krylov_space.results[3] = P*hamiltonian^3 |Krylov_space.results[2]>
# Krylov_space.results[4] = P*hamiltonian^4 |Krylov_space.results[3


## TTN ##
ttn1 = ptn.random_big_ttns_two_root_children()
ham_ttn = ptn.TTNO.from_hamiltonian(ptn.random_hamiltonian_compatible(),ttn1)

Krylov_space = ptn.Krylov(ttn1, ham_ttn, 1,10,  ptn.TensorProduct({"site0": ptn.pauli_matrices()[0]}))
results = Krylov_space.run(4 , ptn.SVDParameters())
print(results[0] == Krylov_space.initial_state)


  0%|          | 0/5 [00:00<?, ?it/s]

NoConnectionException: link_site0_with_site1 is not a neighbour of site0!

# Ortogonalize ttn against ttn 

In [ ]:
"""
I wanted to orthogonalize the outcome at each step to the previous states, but 
I could not find a way to ortogonilize two ttns against each other.
I tried to orthogonalize each tensors seperately, but it did not work.
"""

def orthogonalize_against(ttn1 , ttn2):
    """
    Orthogonalize all locale tensors of ttn1 against ttn2.
    Args:
        ttn1 : TreeTensorNetwork
        ttn2 : TreeTensorNetwork
    """
    for i in range(len(ttn1.nodes)):
        shape = ttn1.tensors[f"site{i}"].shape
        a = ttn1.tensors[f"site{i}"].reshape(-1)
        b = deepcopy(ttn2.tensors[f"site{i}"])
        b = normalize_tensor(b)
        b = b.reshape(-1)
        prod = np.tensordot(b.conj(), a , axes = (0,0) )
        a = a - b * prod
        a = a / np.sqrt(ptn.contract_two_ttns(ttn1,ttn1.conjugate()))
        ttn1.tensors[f"site{i}"] = np.reshape(a,shape)
    return ttn1 , ttn2

def _tensor(tensor): 
    """
      a tensor.
     Args:
          tensor : np.ndarray
     Returns : 
          The normalized tensor.
     """
    indices  = range(tensor.ndim)
    norm = np.sqrt(np.tensordot(tensor,tensor.conj(), axes = (indices , indices) ))
    return tensor / norm
  
## Test : (fails)
shapes = [(5, 2), (5, 7, 2), (7, 3, 2), (3, 6, 2), (6, 30, 2), (30, 2)]
tensors1 = [ptn.crandn(shape) for shape in shapes]
tensors2 = [ptn.crandn(shape) for shape in shapes]
mps1 = ptn.MatrixProductState.from_tensor_list(tensors1,root_site=5,node_prefix="site")
mps2 = ptn.MatrixProductState.from_tensor_list(tensors2,root_site=5,node_prefix="site")
orthogonalize_against(mps1,mps2)

a = mps1.tensors["site2"]
b = mps2.tensors["site2"]
print(np.allclose(np.tensordot(b.conj(), a , axes = ((0,1,2),(0,1,2))), 0))
print(np.allclose(ptn.contract_two_ttns(mps1,mps2.conjugate()),0)) # not orthogonal 

True
False


# Krylov_second_method

In [ ]:
"""
In this method, I contracted hamiltonian(TTNO) to the state(TTN) at each site. 
the bond dimension was growing exponentially, so at each step I transformed
the state to the complete canonical form, where bond dimension are controlled by
SVD truncation parameters.
"""

'\nIn this method, I contracted hamiltonian(TTNO) to the state(TTN) at each site. \nthe bond dimension was growing exponentially, so at each step I transformed\nthe state to the complete canonical form, where bond dimension are controlled by\nSVD truncation parameters.\n'

In [ ]:
# initialize mps : mps1

# Antiferromagnet state
mps_up = ptn.MatrixProductState.constant_product_state(state_value = 0,
                                                     dimension =  2,
                                                     num_sites = 15,
                                                     node_prefix = "site", # default 
                                                     root_site = 14)

mps_down = ptn.MatrixProductState.constant_product_state(state_value = 1,
                                                     dimension =  2,
                                                     num_sites = 15,
                                                     node_prefix = "site", # default 
                                                     root_site = 14)    
mps1 = deepcopy(mps_up)
for i in range(len(mps1.nodes)):
    if i % 2 == 0:
       mps1.tensors[f"site{i}"] = mps_up.tensors[f"site{i}"]
    else:
       mps1.tensors[f"site{i}"] = mps_down.tensors[f"site{i}"]  


# Initialize a random hamiltonian(TTNO) : mpo1

X , Y , Z = ptn.pauli_matrices()
possible_operators = [X, X@Y, Z@X, Z, Y, X@Y , Z@Z, Y@X, Z@Y ] 
sites = [f"site{i}" for i in range(len(mps1.nodes))]
# List = ptn.random_symbolic_terms(num_of_terms=4, 
#                                 possible_operators=possible_operators, 
#                                 sites=sites, 
#                                 min_num_sites=2, 
#                                 max_num_sites=4, 
#                                 seed=None) ----> Error 
###########################################################
List = ptn.random_terms(num_of_terms = 5,
                        possible_operators = possible_operators,
                        sites = sites,
                        min_strength = 1,
                        max_strength = 4,
                        min_num_sites= 10,
                        max_num_sites= 14)
tp_list = [ptn.TensorProduct(term) for term in List]

num_terms = 20
tp_list = [ptn.random_tensor_product(mps1,10,["A","B","C","D"]) for _ in range(num_terms)]
conversion_dictionary = {"A": ptn.random_hermitian_matrix(2),
                         "B": ptn.random_hermitian_matrix(2),
                         "C": ptn.random_hermitian_matrix(2),
                         "D": ptn.random_hermitian_matrix(2),
                         "I2": np.eye(2)}

H = ptn.Hamiltonian(tp_list, conversion_dictionary=conversion_dictionary)
H = H.pad_with_identities(mps1)
ttno1 = ptn.TTNO.from_hamiltonian(H, mps1)
##############################################################
tp1 = ptn.random_tensor_product(mps1, num_operators= len(mps1), possible_operators = possible_operators)
tp2 = ptn.random_tensor_product(mps1, num_operators= len(mps1), possible_operators = possible_operators)

H = ptn.Hamiltonian([tp1,tp2])
H = H.pad_with_identities(mps1)
# mpo1 = ptn.TTNO.from_hamiltonian(H,mps1) ----> Error: StateDiagram.from_hamiltonian
######################################################################################################

conversion_dictionary = {}
for i in range(len(mps1.nodes)):
    conversion_dictionary[f"{i}"] = ptn.random_hermitian_matrix(2)
for i in range(len(mps1.nodes)):
    conversion_dictionary[f"I{i}"] = np.eye(i)

dict = {}
for i in range(len(mps1.nodes)):
    dict[f"site{i}"] = f"{i}"
tp1 = ptn.TensorProduct(dict)

for i in range(len(mps1.nodes)):
    dict[f"site{i}"] = f"{len(mps1.nodes) - i-1}"
tp2 = ptn.TensorProduct(dict)

H = ptn.Hamiltonian([tp1,tp2],conversion_dictionary)
H = H.pad_with_identities(mps1)
mpo1 = ptn.TTNO.from_hamiltonian(H,mps1) # All bond dimnsions are 2 

15


In [ ]:
Krylov_space = ptn.Krylov_second_method(mps1, mpo1)
Krylov_space.run(10, ptn.SVDParameters())
results = Krylov_space.results


# results[0] = initial_state
print(results[0] == mps1)
# results[1] = hamiltonian |inisial_state>
# results[2] = hamiltonian^2 |Krylov_space.results[1]>
# results[3] = hamiltonian^3 |Krylov_space.results[2]>
# results[4] = hamiltonian^4 |Krylov_space.results[3]>
# Check the norms 
print(ptn.contract_two_ttns(results[1],results[1].conjugate()))
print(ptn.contract_two_ttns(results[5],results[5].conjugate()))
print(ptn.contract_two_ttns(results[10],results[10].conjugate()))
# check the orthogonality of root site = "site14"
print(np.allclose(ptn.compute_transfer_tensor( results[2].tensors[ttn1.root_id], (0,)) , np.eye(2) ))
# the growth of bind dimensions at "site10"
print(results[0].tensors["site10"].shape)
print(results[2].tensors["site10"].shape)
print(results[4].tensors["site10"].shape)
print(results[6].tensors["site10"].shape)
print(results[8].tensors["site10"].shape)

100%|██████████| 10/10 [00:00<00:00, 17.56it/s]

True
(0.9999999999999991-6.938893903907228e-18j)
(0.9999999999999991+0j)
(1.0000000000000013+1.3877787807814457e-17j)
True
(1, 1, 2)
(3, 3, 2)
(8, 8, 2)
(21, 21, 2)
(49, 46, 2)


In [ ]:
mps = results[6]

In [ ]:
mps.tensors["site13"].shape

(8, 8, 2)

In [ ]:
mps, _ = mps.complete_canonical_form(ptn.SVDParameters())

# Global Bond expansion

In [ ]:
shapes = [(4,2)] + [(4,4,2)]*13 + [(4,2)]
tensors = [ptn.crandn(shape) for shape in shapes]
mps = ptn.MatrixProductState.from_tensor_list(tensors,root_site=14,node_prefix="site")


In [ ]:
# Prepare the states for bond expansion 
Krylov_space = ptn.Krylov(mps, mpo1, 1, 10, ptn.TensorProduct({"site0": ptn.pauli_matrices()[0]}))
results = Krylov_space.run(20)

mps1 = results[0]
mps2 = results[1]
mps3 = results[2]
mps4 = results[3]


 48%|████▊     | 10/21 [00:00<00:00, 44.84it/s]

100%|██████████| 21/21 [00:00<00:00, 50.38it/s]


In [ ]:
results[3].tensors["site14"].shape

(2, 2)

In [ ]:
U , S , B = ptn.truncated_tensor_svd(mps1.tensors["site14"], (1,), (0,),
                                     max_bond_dim=np.inf , rel_tol=-np.inf, total_tol=-np.inf)
print(U.shape , S.shape , B.shape)

B_dagger = B.conj().T
B_dagger_B = B_dagger @ B 
P = np.eye(B_dagger_B.shape[0]) - B_dagger_B

pho_2 = reduced_density_matrix(mps2,"site14")
pho_3 = reduced_density_matrix(mps3,"site14")
pho_4 = reduced_density_matrix(mps4,"site14")
pho_5 = reduced_density_matrix(mps5,"site14")
pho_6 = reduced_density_matrix(mps6,"site14")
pho_tilde = pho_2 + pho_3 + pho_4 + pho_5 + pho_6

# pho_baar = P @ pho @ P ---> dimension mismatch

TypeError: truncated_tensor_svd() got an unexpected keyword argument 'max_bond_dim'